In [4]:
pip install opencv-python

In [2]:
import face_recognition
#from sklearn import svm
import os
import cv2
import numpy as np

In [3]:
import argparse
import pickle
import csv
from datetime import datetime
    

In [5]:
encodings = []
names = []

# Training directory
train_dir = os.listdir('C:/Users/gourabd838/Documents/Training')

# Loop through each person in the training directory
for person in train_dir:
    pix = os.listdir("C:/Users/gourabd838/Documents/Training/" + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("C:/Users/gourabd838/Documents/Training/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/gourabd838/Documents/Training/desktop.ini'

In [4]:
# serialising the encodings
filename = "face_pkl1.pkl"
data ={"encodings":encodings,"names":names}

if not os.path.isfile(filename):
   with open(filename,'wb') as file:
       pickle.dump(dict, file)
   file.close()

In [18]:
filename = "face_pkl1.pkl"
infile = open(filename,'rb')
dict = pickle.load(infile)
infile.close()
print(dict.values())
def getList(dict): 
    encodings = [] 
    for key in dict.keys(): 
        list.append(key) 
          
    return list
getList(new_dict)

TypeError: descriptor 'values' of 'dict' object needs an argument

In [14]:
def markAttendance(name):
    with open('ATTLOG.csv','r+') as f:
        myDatalist = f.readlines()
        nameList=[]
        for line in myDatalist:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now=datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

In [40]:
def MarkAttendance(name):
    already_in_file = set()
    with open('ATTLOG.csv', "r") as g: 
        # just read
        for line in g:
            already_in_file.add(line.split(",")[0])

# process your current entry:
    if name not in already_in_file:
        with open('ATTLOG.csv', "a") as g:
            # append
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            g.writelines(f'\n{name},{dtString}')

In [4]:
# Initialize some variables
video_capture = cv2.VideoCapture(0)
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        markAttendance(name)
        
  

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

NameError: name 'markAttendance' is not defined

In [39]:
with open('ATTLOG.csv', "r+") as g:
            myDatalist = g.readlines()
            nameList=[]
            for line in myDatalist:
                entry = line.split(',')
                nameList.append(entry[0]
            print(*nameList, sep = ", ")               

SyntaxError: invalid syntax (<ipython-input-39-abdd2cc909f9>, line 7)

In [36]:
with open('ATTLOG.csv','r') as f:
            myDatalist = f.readlines()
            nameList=[]
            for line in myDatalist:
                entry = line.split(',')
                nameList.append(entry[0])
                if name not in nameList:

SyntaxError: unexpected EOF while parsing (<ipython-input-36-852ec99f279c>, line 7)

In [ ]:
with open('ATTLOG.csv', "r+") as g:
            myDatalist = g.readlines()
            nameList=[]
            for line in myDatalist:
                entry = line.split(',')
                nameList.append(entry[0])
                if name not in nameList:
                    now=datetime.now()
                    dtString = now.strftime('%H:%M:%S')
                    g.writelines(f'\n{name},{dtString}')

In [20]:
import csv
from datetime import datetime
    
with open("ATTLOGERR.csv", "w") as my_empty_csv:
    pass

ValueError: must have exactly one of create/read/write/append mode

In [ ]:
with open("ATTLOG.csv", 'w', newline='') as file:
            now=datetime.now()
            writer = csv.writer(file)
            dtString = now.strftime('%H:%M:%S')
            #writer.writerow({name})
            file.writelines(f'\n{name},{dtString}')

In [35]:
import csv
with open("ATTLOG.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    dtString = now.strftime('%H:%M:%S')
    writer.writerow([name],{dtstring})

SyntaxError: invalid syntax (<ipython-input-35-3fb6d1a981e1>, line 6)

In [ ]:
import csv
data_list = [["SN", "Name", "Time"],
             [1, "Linus Torvalds", "Linux Kernel"],
             [2, "Tim Berners-Lee", "World Wide Web"],
             [3, "Guido van Rossum", "Python Programming"]]
with open('ATTLOG.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerows(data_list)